In [1]:
from langchain_community.retrievers import BM25Retriever
from typing import List
import jieba
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel, AutoTokenizer

/root/miniconda3/envs/easyr1/lib/python3.10/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/root/miniconda3/envs/easyr1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = TextLoader('/root/autodl-tmp/LLM_github/medical_data.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    separators=['\n']
)
docs = text_splitter.split_documents(documents)

In [5]:
len(docs)
docs

[Document(metadata={'source': '/root/autodl-tmp/LLM_github/medical_data.txt'}, page_content="{'question': '曲匹地尔片的用法用量', 'answer': '注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。'}"),
 Document(metadata={'source': '/root/autodl-tmp/LLM_github/medical_data.txt'}, page_content="\n{'question': '三期梅毒多久能治愈吗', 'answer': '梅毒是一种进展十分缓慢的疾病，包括早期梅毒，晚期梅毒等等，一般来说，梅毒病程超过两年的就是晚期梅毒了，而晚期梅毒的治疗难度是比较大的，三期梅毒能完全治愈吗?该如何进行治疗?一、对于三期梅毒的治疗，中西各有妙招，希望患者在医生的指导下选择适合自己的治疗方法。被确诊为三期梅毒的患者很关心这个问题，梅毒需要讲究科学的治疗方法，三期梅毒患者如果有一个良好的心态，积极配合医生治疗，还有很有可能治愈的，具体治疗方法我们看看下文的介绍。二、三期梅毒又称晚期梅毒，是因梅毒初期没有进行治疗或者治疗不彻底所造成。好发于40-50岁之间。该期梅毒不仅局限于皮肤和粘膜，也可侵犯任何内脏器官和组织。三期梅毒可发生在感染后2年以上，一般多发于感染后3～4年。病程漫长，可持续10～30年。未经治愈的二期梅毒中约有1/3的病人可发展为晚期活动性梅毒;另有一部分患者不出现晚期梅毒症状，只是梅毒血清反应持续阳性，为晚期潜伏梅毒;也有一部分患者可以自愈。三、晚期梅毒可以通过药物治疗控制病情，但是治愈的可能性是比较小的，治疗还是有必要的，通过有效的治疗可以缓解患者的症状，杀灭一部分的梅毒螺旋体。虽然治愈的可能性不高，但是患者还是应当坚持治疗。通过上文的简单介绍，相信大家对于三期梅毒多久能治愈这一方面的问题已经有了自己的答案。对于梅毒患者来说，无论患病多少年，都应在确诊后及时进行正规的治疗处理。正规的治疗可以缓解患者的病情，患者还是应当坚持进行，并积极配合医生的治疗，说不定治愈的可能就发生了。'}"),


In [10]:
def preprocessing_func(text: str) -> List[str]:
    return list(jieba.cut(text))
bm25 = BM25Retriever(docs=docs)
print(bm25.k)
retriever = bm25.from_documents(docs,preprocess_func=preprocessing_func)

4


In [11]:
retriever.invoke('骨折了应该怎么办')

[Document(metadata={'source': '/root/autodl-tmp/LLM_github/medical_data.txt'}, page_content="\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}"),
 Document(metadata={'source': '/root/autodl-tmp/LLM_github/medical_data.txt'}, page_content="\n{'question': '手腕骨折一直肿胀怎么办', 'answer': '手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。白天，受影响的上肢可以挂在脖子上。每天局部敷热敷。干燥后，尝试外部消毒。同时，你可以饮用活血化瘀的药物。如果是风湿性疾病，要

In [12]:
from rank_bm25 import BM25Okapi
texts = [i.page_content for i in docs]
texts_processed = [preprocessing_func(t) for t in texts]
vectorizer = BM25Okapi(texts_processed)

In [14]:
for i in docs :
    print(i.page_content)
    print(preprocessing_func(i.page_content))
    break

{'question': '曲匹地尔片的用法用量', 'answer': '注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。'}
['{', "'", 'question', "'", ':', ' ', "'", '曲匹', '地尔片', '的', '用法', '用量', "'", ',', ' ', "'", 'answer', "'", ':', ' ', "'", '注意', '：', '同种', '药品', '可', '由于', '不同', '的', '包装', '规格', '有', '不同', '的', '用法', '或', '用量', '。', '本文', '只', '供参考', '。', '如果', '不', '确定', '，', '请', '参看', '药品', '随带', '的', '说明书', '或', '向', '医生', '询问', '。', '口服', '。', '一次', '50', '～', '100mg', '（', '1', '-', '2', '片', '）', '，', '3', '次', '/', '日', '，', '或', '遵', '医嘱', '。', "'", '}']


In [15]:
vectorizer.get_top_n(preprocessing_func('骨折了应该怎么办'),texts, n=10)

["\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}",
 "\n{'question': '手腕骨折一直肿胀怎么办', 'answer': '手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。白天，受影响的上肢可以挂在脖子上。每天局部敷热敷。干燥后，尝试外部消毒。同时，你可以饮用活血化瘀的药物。如果是风湿性疾病，要根据检查结果选择治疗合适的治疗方法的。手腕骨折在日常生活中较为常见，大部分受伤者是老年人，大部分患者是由于手掌落地后摔倒造成的。大多数骨折发生在桡骨远端关节附近2厘米处，临床上称为科利尔骨折。没有及时有效的治疗，患者的腕关节会出现关节畸形、关节运动功能受限、关节疼痛等症状，给患者的日常生活带来诸多不便。因此，在诊断和治疗中必须达到正确的复位和良好的固定。骨折初期肿

In [18]:
model_name = "BAAI/bge-large-zh-v1.5"
model = AutoModel.from_pretrained(model_name)
embeddings = AutoTokenizer.from_pretrained(model_name)

# embeddings = HuggingFaceEmbeddings(model_name='/root/autodl-tmp/LLM_github/bge-large-zh-v1.5', model_kwargs = {'device': 'cuda:0'})

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
bm25_res = vectorizer.get_top_n(preprocessing_func('骨折了应该怎么办'),texts, n=10)
bm25_res

In [ ]:
vector_res = db.similarity_search('骨折了应该怎么办', k=10)
vector_res

In [ ]:
def rrf(vector_results: List[str], text_results: List[str], k: int=10, m: int=60):
        """
        使用RRF算法对两组检索结果进行重排序
        
        params:
        vector_results (list): 向量召回的结果列表,每个元素是专利ID
        text_results (list): 文本召回的结果列表,每个元素是专利ID
        k(int): 排序后返回前k个
        m (int): 超参数
        
        return:
        重排序后的结果列表,每个元素是(文档ID, 融合分数)
        """
        
        doc_scores = {}
        
        # 遍历两组结果,计算每个文档的融合分数
        for rank, doc_id in enumerate(vector_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)
        for rank, doc_id in enumerate(text_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)
        
        # 将结果按融合分数排序
        sorted_results = [d for d, _ in sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:k]]

        return sorted_results

In [ ]:
vector_results = [i.page_content for i in vector_res]
text_results = [i for i in bm25_res]
rrf_res = rrf(vector_results, text_results)
rrf_res

In [ ]:
prompt = '''
任务目标：根据检索出的文档回答用户问题
任务要求：
    1、不得脱离检索出的文档回答问题
    2、若检索出的文档不包含用户问题的答案，请回答我不知道

用户问题：
{}

检索出的文档：
{}
'''

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='Qwen2-7B-Instruct', base_url='http://localhost:8000/v1', api_key='n')
res = model.invoke(prompt.format('骨折了应该怎么办', ''.join(rrf_res)))
print(res.content)

In [ ]:
res = model.invoke('骨折了应该怎么办')
print(res.content)